In [10]:
from dotenv import load_dotenv

load_dotenv()

True

## Pinecone

Pinecone은 고성능 벡터 데이터베이스로, AI 및 머신러닝 애플리케이션을 위한 효율적인 벡터 저장 및 검색 솔루션입니다.


주요 특징:

1. 벡터 데이터 저장 및 검색: 고차원 벡터 데이터를 효율적으로 저장하고 검색할 수 있는 시스템을 제공합니다.

2. 확장성: 대규모 데이터셋을 처리할 수 있는 확장 가능한 아키텍처를 갖추고 있습니다.

3. 개발자 친화적: API를 통해 쉽게 사용할 수 있으며, 다양한 AI 모델과 호환됩니다.

4. 완전 관리형 서비스: 사용자가 인프라 관리에 신경 쓰지 않고 애플리케이션 개발에 집중할 수 있습니다.

주요 장점:

1. 고성능 검색: 수십억 개의 아이템을 밀리초 단위로 빠르게 검색할 수 있습니다.

2. AI 애플리케이션 강화: 관련 정보 검색에 의존하는 AI 애플리케이션의 성능을 향상시킬 수 있습니다.

3. 쉬운 시작 및 확장: 몇 번의 클릭이나 API 호출만으로 계정 생성과 인덱스 설정이 가능합니다.

In [2]:
# !pip install -U langchain-teddynote

  Attempting uninstall: langchain-teddynote
    Found existing installation: langchain-teddynote 0.0.11
    Uninstalling langchain-teddynote-0.0.11:
      Successfully uninstalled langchain-teddynote-0.0.11


In [3]:
from langchain_teddynote.korean import stopwords

# 한글 불용어 사전 불러오기 (불용어 사전 출처: https://www.ranks.nl/stopwords/korean)
stopwords()[:20]

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다']

### 데이터 전처리

In [5]:
import glob
from langchain.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


ROOT_DIR = "./data"
files = glob.glob(ROOT_DIR + "/*.pdf")

document_list = []
for file in files:
    docs = PyMuPDFLoader(file).load()
    document_list.extend(docs)

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=50)
split_docs = text_splitter.split_documents(document_list)

In [7]:
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs=split_docs, metadata_keys=["source", "page"]
)

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 733/733 [00:00<00:00, 733053.13it/s]


### Pinecone: 인덱스 생성

In [24]:
import os
from langchain_teddynote.community.pinecone import create_index
from pinecone import Pinecone, ServerlessSpec
import pinecone

# Pinecone 클라이언트 생성
pinecone_client = Pinecone(
    api_key=os.environ["PINECONE_API_KEY"],
    environment="us-west-2"  # 사용하려는 Pinecone 환경으로 바꾸세요
)

# 인덱스 이름
index_name = "teddynote-db-index"

# 인덱스가 존재하는지 확인하고 삭제
if index_name in [index.name for index in pinecone_client.list_indexes()]:
    pinecone_client.delete_index(index_name)
    print(f"Deleted existing index: {index_name}")

pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name=index_name,  # 인덱스 이름을 지정합니다.
    dimension=1024,  # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
    metric="dotproduct",  # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine)
)

Deleted existing index: teddynote-db-index
[create_index]
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}


In [25]:
from langchain_teddynote.community.pinecone import (
    create_sparse_encoder,
    fit_save_sparse_encoder,
)

# 한글 불용어 사전 + Kiwi 형태소 분석기를 사용합니다.
sparse_encoder = create_sparse_encoder(stopwords(), mode="kiwi")

In [26]:
# Sparse Encoder 를 학습합니다.
# 저장하는 경로(save_path)를 잘 기억합니다.
saved_path = fit_save_sparse_encoder(
    sparse_encoder=sparse_encoder, contents=contents, save_path="./sparse_encoder.pkl"
)

100%|██████████| 733/733 [00:06<00:00, 111.29it/s]

[fit_save_sparse_encoder] Saved sparse encoder to ./sparse_encoder.pkl


In [27]:
from langchain_teddynote.community.pinecone import load_sparse_encoder

# 추후에 학습된 sparse encoder 를 불러올 때 사용합니다.
sparse_encoder = load_sparse_encoder("./sparse_encoder.pkl")

[load_sparse_encoder] load sparse encoder from ./sparse_encoder.pkl


#### Pinecone: DB Index에 추가 (Upsert)


- context: 문서의 내용입니다.
- page: 문서의 페이지 번호입니다.
- source: 문서의 출처입니다.
- values: Embedder 를 통해 얻은 문서의 임베딩입니다.
- sparse values: Sparse Encoder 를 통해 얻은 문서의 임베딩입니다.

In [28]:
from langchain_teddynote.community.pinecone import upsert_documents
from langchain.embeddings import HuggingFaceEmbeddings

# HuggingFaceHubEmbeddings 인스턴스를 생성합니다.
embedder = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    model_kwargs={"device": "cpu"},  # 'cuda'로 설정할 수 있음
    encode_kwargs={"normalize_embeddings": True},
)

# 문서를 Pinecone에 업로드합니다.
upsert_documents(
    index=pc_index,  # Pinecone 인덱스
    namespace="teddynote-namespace-01",  # Pinecone namespace
    contents=contents,  # 이전에 전처리한 문서 내용
    metadatas=metadatas,  # 이전에 전처리한 문서 메타데이터
    sparse_encoder=sparse_encoder,  # Sparse encoder
    embedder=embedder,  # Dense Embedder
)

100%|██████████| 46/46 [08:26<00:00, 11.02s/it]


[upsert_documents]
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 733}},
 'total_vector_count': 733}


### 조회

In [32]:
from langchain_teddynote.community.pinecone import init_pinecone_index

pinecone_params = init_pinecone_index(
    index_name="teddynote-db-index",  # Pinecone 인덱스 이름
    namespace="teddynote-namespace-01",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_pkl_path="./sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=embedder,  # Dense Embedder
    top_k=3,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

[init_pinecone_index]
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 733}},
 'total_vector_count': 733}


In [33]:
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever

pinecone_retriever = PineconeKiwiHybridRetriever(**pinecone_params)

In [34]:
# 실행 결과
search_results = pinecone_retriever.invoke("앤스로픽")
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

namespace teddynote-namespace-01
論
語
논
어
{'page': 11.0, 'source': './data\\1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf', 'score': 0.43457744}


291
周
易
주
역
{'page': 291.0, 'source': './data\\1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf', 'score': 0.4279912}


257
書
經
서
경
{'page': 257.0, 'source': './data\\1568)누구나 한번쯤 읽어야 할 사서삼경 (미리내공방) .pdf', 'score': 0.42739013}


